In [22]:
import pandas as pd

# Load dataset
df = pd.read_csv("Telco_Customer_Churn.csv")

# Drop customerID (not useful)
df.drop("customerID", axis=1, inplace=True)

# Convert TotalCharges to numeric
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df.dropna(inplace=True)

# Target variable
y = df["Churn"].map({"Yes": 1, "No": 0})
X = df.drop("Churn", axis=1)

print("Data shape:", X.shape, y.shape)
print("Features:", X.columns.tolist())



Data shape: (7032, 19) (7032,)
Features: ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']


Step 2: Train/Test Split and Baseline Models

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print("Train:", X_train.shape, " Test:", X_test.shape)


Train: (5625, 19)  Test: (1407, 19)


In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Preprocessing
numeric_features = ["tenure", "MonthlyCharges", "TotalCharges"]
categorical_features = [col for col in X.columns if col not in numeric_features]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

# Pipeline with Logistic Regression
log_reg = LogisticRegression(max_iter=2000, solver="liblinear")

pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", log_reg)
])

# Hyperparameter tuning
param_grid = {
    "classifier__C": [0.01, 0.1, 1, 5, 10],
    "classifier__penalty": ["l1", "l2"]
}

grid = GridSearchCV(pipeline, param_grid, cv=5, scoring="accuracy", n_jobs=-1)
grid.fit(X_train, y_train)

# Best pipeline
final_pipeline = grid.best_estimator_
print("Best Parameters:", grid.best_params_)
print("Best CV Accuracy:", grid.best_score_)

# Evaluate on internal test
y_pred = final_pipeline.predict(X_test)
print("\nInternal Test Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

# Save pipeline
joblib.dump(final_pipeline, "best_model.joblib")
print("Saved: best_model.joblib")


Best Parameters: {'classifier__C': 0.01, 'classifier__penalty': 'l2'}
Best CV Accuracy: 0.8046222222222223

Internal Test Performance:
Accuracy: 0.7967306325515281
Precision: 0.6486486486486487
Recall: 0.5133689839572193
F1 Score: 0.573134328358209
Saved: best_model.joblib


In [25]:
# Load external test dataset
test_df = pd.read_excel("TEST_DATA.xlsx")

# Clean same as training
test_df.drop("customerID", axis=1, inplace=True)
test_df["TotalCharges"] = pd.to_numeric(test_df["TotalCharges"], errors="coerce")
test_df.dropna(inplace=True)

# Features & target
y_test_ext = test_df["Churn"].map({"Yes": 1, "No": 0})
X_test_ext = test_df.drop("Churn", axis=1)

# Load pipeline
pipeline = joblib.load("best_model.joblib")

# Predict
y_pred_ext = pipeline.predict(X_test_ext)

# Metrics
print("\nExternal Test Performance:")
print("Accuracy:", accuracy_score(y_test_ext, y_pred_ext))
print("Precision:", precision_score(y_test_ext, y_pred_ext))
print("Recall:", recall_score(y_test_ext, y_pred_ext))
print("F1 Score:", f1_score(y_test_ext, y_pred_ext))



External Test Performance:
Accuracy: 0.54
Precision: 0.6666666666666666
Recall: 0.0425531914893617
F1 Score: 0.08



Results Data Summary

In [26]:
results_data = [
    {
        "Dataset": "Internal Test",
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred),
        "Best Parameters": grid.best_params_
    },
    {
        "Dataset": "External Test",
        "Accuracy": accuracy_score(y_test_ext, y_pred_ext),
        "Precision": precision_score(y_test_ext, y_pred_ext),
        "Recall": recall_score(y_test_ext, y_pred_ext),
        "F1 Score": f1_score(y_test_ext, y_pred_ext),
        "Best Parameters": grid.best_params_
    }
]

pd.DataFrame(results_data).to_excel("results_summary.xlsx", index=False)
print("Results saved: results_summary.xlsx")


Results saved: results_summary.xlsx
